## Install Emoji

`pip install emoji`

In [0]:
!pip install emoji

## Using Emoji

In [0]:
import emoji
emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":grinning_face_with_big_eyes:",
                    "3": ":zipper-mouth_face:",
                    "4": ":fork_and_knife:",
                    "5": ":tongue:",
                    "6": ":sunglasses:",
                    "7": ":kiss:",
                    "8": ":winking_face_with_tongue:",
                   }

In [0]:
for key in emoji_dictionary.keys() :
    print(emoji.emojize(emoji_dictionary[key]))

❤️
⚾
😃
🤐
🍴
👅
🕶
💏
😜


In [0]:
# ":grinning_face_with_big_eyes:")) 
# print(emoji.emojize(":winking_face_with_tongue:")) 
# print(emoji.emojize(":zipper-mouth_face:")) 

In [0]:
import numpy as np
import pandas as pd

In [0]:
train = pd.read_csv('/content/train_emoji.csv', header = None)

In [0]:
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [0]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       132 non-null    object 
 1   1       132 non-null    int64  
 2   2       0 non-null      float64
 3   3       12 non-null     object 
dtypes: float64(1), int64(1), object(2)
memory usage: 4.2+ KB


In [0]:
X_train = train.iloc[:, 0].values
y_train = train.iloc[:, 1].values

In [0]:
X_train[:10]

array(['never talk to me again', 'I am proud of your achievements',
       'It is the worst day in my life', 'Miss you so much',
       'food is life', 'I love you mum', 'Stop saying bullshit',
       'congratulations on your acceptance',
       'The assignment is too long ', 'I want to go play'], dtype=object)

In [0]:
for i in range(10) :
    print(X_train[i], end = " ")
    print(emoji.emojize(emoji_dictionary[str(y_train[i])])) 

never talk to me again 🤐
I am proud of your achievements 😃
It is the worst day in my life 🤐
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit 🤐
congratulations on your acceptance 😃
The assignment is too long  🤐
I want to go play ⚾


In [0]:
from sklearn.preprocessing import OneHotEncoder
one = OneHotEncoder()
y_train = one.fit_transform(y_train.reshape(-1, 1))
y_train.shape

(132, 5)

In [0]:
! pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"smagni","key":"0ea22017b6052eb6b8adc0ef5988d4dc"}'}

In [0]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [0]:
!kaggle datasets download -d watts2/glove6b50dtxt

 93% 63.0M/67.7M [00:01<00:00, 21.1MB/s]
100% 67.7M/67.7M [00:01<00:00, 38.0MB/s]


In [0]:
!unzip /content/glove6b50dtxt.zip

Archive:  /content/glove6b50dtxt.zip
  inflating: glove.6B.50d.txt        


In [0]:
emb = {}
with open('/content/glove.6B.50d.txt', encoding = 'utf-8') as f :
    for line in f:
        # print(line)
        values = line.split()
        word = values[0]
        coffe = np.asarray(values[1:], dtype= 'float32')
        emb[word] = coffe
    # print(f)

In [0]:
def togetOutputEMB(X) :
    emb_mat_output = np.zeros((X.shape[0], 10, 50))

    for ix in range(X.shape[0]) :
        X[ix] = X[ix].split()
        for ij in range(len(X[ix])) :
            emb_mat_output[ix][ij] = emb[X[ix][ij].lower()]
    
    return emb_mat_output

In [0]:
X_train = togetOutputEMB(X_train)

In [0]:
X_train.shape

(132, 10, 50)

In [0]:
from keras import Sequential
from keras.layers import *

In [0]:
model = Sequential()
model.add(LSTM(64, input_shape = (10, 50), return_sequences= True))
model.add(Dropout(rate = 0.5))
model.add(LSTM(64, return_sequences= False))
model.add(Dropout(rate = 0.3))
model.add(Dense(5, activation= 'softmax'))

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 64)            29440     
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 325       
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer= 'rmsprop', loss = 'categorical_crossentropy', metrics= ['accuracy'])

In [0]:
model.fit(X_train, y_train, epochs= 10, batch_size= 32, shuffle= True, validation_split= 0.1)

Train on 118 samples, validate on 14 samples
Epoch 1/10
118/118 [==============================] - 3s 21ms/step - loss: 1.5791 - accuracy: 0.2203 - val_loss: 1.6176 - val_accuracy: 0.3571
Epoch 2/10
118/118 [==============================] - 0s 2ms/step - loss: 1.5020 - accuracy: 0.3136 - val_loss: 1.6015 - val_accuracy: 0.3571
Epoch 3/10
118/118 [==============================] - 0s 2ms/step - loss: 1.4456 - accuracy: 0.3475 - val_loss: 1.5475 - val_accuracy: 0.4286
Epoch 4/10
118/118 [==============================] - 0s 2ms/step - loss: 1.3530 - accuracy: 0.4322 - val_loss: 1.4485 - val_accuracy: 0.3571
Epoch 5/10
118/118 [==============================] - 0s 2ms/step - loss: 1.2439 - accuracy: 0.4915 - val_loss: 1.4300 - val_accuracy: 0.5714
Epoch 6/10
118/118 [==============================] - 0s 2ms/step - loss: 1.1406 - accuracy: 0.6102 - val_loss: 1.3313 - val_accuracy: 0.4286
Epoch 7/10
118/118 [==============================] - 0s 2ms/step - loss: 1.0528 - accuracy: 0.6525 - 

In [0]:
model.evaluate(X_train, y_train)

132/132 [==============================] - 0s 413us/step


[0.7680458408413511, 0.75]

In [0]:
y_pred = model.predict(X_train)

In [0]:
for i in range(y_pred.shape[0]) :
    # print(np.argmax(y_pred[i]))
    print(emoji.emojize(emoji_dictionary[str(np.argmax(y_pred[i]))]) , end = " ")
    print(emoji.emojize(emoji_dictionary[str(np.argmax(y_train[i]))]))

🤐 🤐
😃 😃
😃 🤐
😃 ❤️
🍴 🍴
❤️ ❤️
🤐 🤐
😃 😃
😃 🤐
⚾ ⚾
🤐 🤐
🤐 🤐
⚾ ⚾
😃 🤐
🤐 😃
🤐 🤐
😃 😃
😃 🤐
⚾ ⚾
🍴 😃
🤐 🤐
❤️ ❤️
😃 😃
😃 😃
❤️ 😃
⚾ ⚾
😃 🍴
😃 😃
🤐 😃
🍴 🍴
😃 ❤️
🤐 🤐
🍴 🍴
😃 😃
😃 ❤️
🍴 🤐
😃 😃
😃 😃
🤐 🤐
🍴 🍴
😃 😃
😃 😃
❤️ ❤️
😃 😃
😃 🤐
❤️ ❤️
🤐 🤐
😃 😃
🍴 🍴
⚾ 🤐
🤐 ❤️
🤐 🤐
🤐 🤐
❤️ 🤐
🍴 🍴
😃 😃
🍴 ⚾
⚾ ⚾
⚾ ⚾
😃 😃
🤐 🤐
⚾ ⚾
😃 ❤️
😃 ❤️
❤️ ❤️
🤐 🤐
🍴 🍴
🍴 🍴
😃 😃
😃 😃
⚾ ⚾
😃 😃
❤️ ❤️
🤐 🤐
😃 😃
😃 😃
❤️ ❤️
😃 ❤️
🤐 🤐
⚾ ⚾
😃 😃
⚾ ⚾
😃 😃
😃 😃
🍴 🍴
😃 🤐
🤐 🤐
😃 😃
🍴 🍴
😃 ❤️
❤️ ❤️
😃 ❤️
😃 🤐
🤐 🤐
🤐 🤐
😃 😃
🤐 ❤️
⚾ ⚾
😃 😃
🤐 🤐
❤️ ❤️
😃 😃
😃 😃
😃 😃
🤐 🤐
😃 😃
😃 😃
😃 😃
🍴 🍴
⚾ ⚾
⚾ ⚾
😃 🤐
🤐 🤐
🍴 🍴
⚾ ⚾
😃 😃
⚾ ⚾
⚾ ⚾
😃 🤐
⚾ ⚾
😃 ❤️
🍴 🍴
😃 ❤️
🤐 🤐
😃 🤐
🍴 🍴
😃 🍴
⚾ ⚾
🍴 🍴
😃 🤐
❤️ ❤️
😃 😃
